# Azure AI Search 與 NVIDIA NIM 和 LlamaIndex 整合

在這份筆記中，我們將展示如何利用 NVIDIA 的 AI 模型和 LlamaIndex，建立一個強大的檢索增強生成 (RAG) 管道。我們將使用 NVIDIA 的大型語言模型 (LLM) 和嵌入技術，將它們與 Azure AI Search 整合作為向量存儲，並執行 RAG 以提升搜尋的品質和效率。

## 優勢
- **可擴展性**：利用 NVIDIA 的大型語言模型和 Azure AI Search，實現可擴展且高效的檢索。
- **成本效益**：透過高效的向量存儲和混合搜尋技術，優化搜尋和檢索成本。
- **高效能**：結合強大的 LLM 和向量化搜尋，提供更快速且更準確的回應。
- **高品質**：透過相關檢索文件，確保 LLM 回應的品質。

## 先決條件
- 🐍 Python 3.9 或更高版本
- 🔗 [Azure AI Search Service](https://learn.microsoft.com/azure/search/)
- 🔗 NVIDIA API 金鑰，用於透過 NVIDIA NIM 微服務存取 NVIDIA 的 LLM 和嵌入技術

## 涵蓋的功能
- ✅ NVIDIA LLM 整合（我們將使用 [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe)）
- ✅ NVIDIA 嵌入技術（我們將使用 [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5)）
- ✅ Azure AI Search 高級檢索模式
- ✅ 使用 LlamaIndex 進行文件索引
- ✅ 使用 Azure AI Search 和 LlamaIndex 與 NVIDIA LLM 執行 RAG

讓我們開始吧！


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## 安裝與需求
使用 Python 版本 >3.10 建立 Python 環境。

## 快速開始！


要開始使用 NVIDIA AI Foundation 模型，您需要一個 `NVIDIA_API_KEY`：
1) 建立一個 [NVIDIA](https://build.nvidia.com/explore/discover) 的免費帳戶。
2) 點選您選擇的模型。
3) 在輸入區域，選擇 Python 標籤，然後點擊 **Get API Key**，接著點擊 **Generate Key**。
4) 複製並保存生成的金鑰作為 NVIDIA_API_KEY。從這裡開始，您就可以使用端點了。


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## RAG 範例使用 LLM 和嵌入
### 1) 初始化 LLM
`llama-index-llms-nvidia`，也就是 NVIDIA 的 LLM 連接器，允許您連接並從 NVIDIA API 目錄中可用的兼容模型生成內容。請參閱此處以獲取聊天完成模型的列表：https://build.nvidia.com/search?term=Text-to-Text

在這裡，我們將使用 **mixtral-8x7b-instruct-v0.1**


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) 初始化嵌入

`llama-index-embeddings-nvidia`，也被稱為 NVIDIA 的嵌入連接器，允許您連接並從 NVIDIA API 目錄中可用的兼容模型生成。我們選擇了 `nvidia/nv-embedqa-e5-v5` 作為嵌入模型。請參閱此處以獲取文本嵌入模型的列表：https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

### 3) 建立 Azure AI 搜索向量存儲


In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) 建立一個查詢引擎以便對您的數據進行提問

以下是一個使用 Azure AI Search 的純向量搜索進行查詢，並將回應基於我們的 LLM (Phi-3.5-MOE)。


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

以下是使用混合搜索在 Azure AI Search 中的查詢。


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### 向量搜索分析
LLM 的回應準確地捕捉了來源文本中提到的俄羅斯股市的主要經濟影響。具體而言，它指出俄羅斯股市出現了大幅下跌，市值損失了 40%，並且因為當前局勢而暫停交易。這一回應與來源提供的信息高度一致，表明 LLM 正確地識別並總結了俄羅斯行動及制裁所導致的股市影響相關細節。

#### 來源節點評論
來源節點詳細描述了俄羅斯因國際制裁而面臨的經濟後果。文本強調俄羅斯股市市值損失了 40%，並且交易被暫停。此外，還提到其他經濟影響，例如盧布貶值以及俄羅斯經濟的更廣泛孤立。LLM 的回應有效地提取了這些節點中的關鍵點，並根據查詢要求聚焦於股市影響。


現在，讓我們來看看一個混合搜索無法提供充分答案的查詢：


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### 混合搜索：LLM 回應分析
在混合搜索範例中，LLM 的回應指出提供的內容並未明確說明俄羅斯入侵烏克蘭的確切日期。此回應表明 LLM 正在利用來源文件中的資訊，但同時承認文本中缺乏精確細節。

該回應準確地指出內容提到了與俄羅斯侵略相關的事件，但未具體指出入侵的日期。這展示了 LLM 能夠理解所提供的資訊，同時識別內容中的空白。LLM 有效地提示使用者尋求外部來源或歷史記錄以獲得確切日期，展現出在資訊不完整時的謹慎態度。

### 來源節點分析
混合搜索範例中的來源節點包含一段演講摘錄，討論美國對俄羅斯在烏克蘭行動的回應。這些節點強調了更廣泛的地緣政治影響，以及美國及其盟友對入侵所採取的措施，但並未提到具體的入侵日期。這與 LLM 的回應一致，後者正確地指出內容中缺乏精確的日期資訊。


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### 混合模式搭配重新排序：LLM回應分析
在混合模式搭配重新排序的範例中，LLM的回應透過指出事件發生在演講前六天，提供了額外的背景資訊。這表明LLM能夠根據演講的時間推斷出入侵日期，儘管仍需知道演講的確切日期才能達到精準。

此回應展現了利用上下文線索提供更具資訊性的答案的能力提升。它突顯了重新排序的優勢，讓LLM能夠存取並優先處理更相關的資訊，以更接近所需細節（例如入侵日期）的答案。

### 資源節點分析
此範例中的資源節點包括提到俄羅斯入侵時間的參考，特別指出事件發生在演講前六天。雖然仍未明確指出確切日期，但這些節點提供了時間背景，使LLM能夠給出更細緻的回應。這項細節的加入展示了重新排序如何提升LLM從提供的上下文中提取並推斷資訊的能力，進而產生更準確且具資訊性的回應。


**注意：**  
在此筆記本中，我們使用了來自 NVIDIA API Catalog 的 NVIDIA NIM 微服務。  
上述 API，包括 `NVIDIA (llms)`、`NVIDIAEmbedding` 和 [Azure AI Search Semantic Hybrid Retrieval（內建重排序功能）](https://learn.microsoft.com/azure/search/semantic-search-overview)。請注意，上述 API 也支持自託管的微服務。

**範例：**  
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")```



---

**免責聲明**：  
本文件已使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。儘管我們致力於提供準確的翻譯，請注意自動翻譯可能包含錯誤或不準確之處。原始文件的母語版本應被視為權威來源。對於關鍵資訊，建議使用專業人工翻譯。我們對因使用此翻譯而引起的任何誤解或錯誤解釋不承擔責任。
